Key metrics to monitor success of team's effort in improving the guest host matching process?

Goal of the company:
- increase bookings on platform

Ideas:
- contact to booking ratio
- listings to booking ratio
- interactions before booking

## Import Required Python Packages and Define Functions

In [1]:
# Import required packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
def dataframe_overview(df,df_name):
    '''
    This function presents an overview of the DataFrame
    
    Input
    -----
    df: a DataFrame object
    df_name: a string containing the name of the DataFrame object
    
    Output
    ------
    Print out overview for provided DataFrame
    '''
    
    # Define variables to use in print out
    # Number of null values
    null_cells = df.isna().sum().sum()
    # Number of rows that contain null values in any column
    null_rows = df.isna().any(axis = 1).sum()
    # Number of cells (rows in DataFrame multiplied by columns in DataFrame)
    df_cells = df.shape[0] * df.shape[1]
    
    # Apply variables in print statements
    print(f"Overview statistics for {df_name}:")
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")
    
    print(f"Number of rows with null values in any column: {null_rows}")
    print(f"% of rows with null values in any column: {np.round(null_rows/df.shape[0] *100,2)}%")
    
    print(f"Number of null cells: {null_cells}")
    print(f"% of cells with null values: {np.round(null_cells/df_cells *100,2)}%")

## Importing Data

### Main data table

**Contacts.csv**, in which each row represents an inquiry for a stay at a listing in Rio de Janeiro, was loaded into the DataFrame **contacts_df**. Contacts_df was identified as the main table (central table within a star schema) as it contains information regarding each inquiry, which is essential to measuring the performance of the team in Rio de Janeiro and AirBnB in general. This is because the conversion of an inquiry into a booking is what generates income for AirBnB.

In [3]:
# contacts_df: contains a row for everytime a user makes an inquiry for a stay at a listing in Rio de Janeiro
contacts_df = pd.read_csv(
    'data/contacts.csv', 
    # The columns that are datetime or date objects based on provided data dictionary were passed into the parse_dates argument
    parse_dates = [
        'ts_interaction_first', # UTC timestamp of the moment the inquiry is made
        'ts_reply_at_first', # UTC timestamp of the moment host replies to inquiry, if so
        'ts_accepted_at_first', # UTC timestamp of the momentt host accepts the inquiry, if so
        'ts_booking_at', # UTC timestamp of the moment the booking is made, if so
        'ds_checkin_first', # timestamp of the checkin date of inquiry
        'ds_checkout_first' # timestamp of the checkout date of inquiry
    ]
)

The data available contained a total of 27,887 inquiries. Although 58.45% of the inquiries contained a null value, the null value may indicate an inquiry that resulted in no booking and requires further data exploration. To affirm the previous statement, only 7.63% of all cells were null, further supporting the need to explore the columns deeply.

In [14]:
# Examine the overview for the loaded data
dataframe_overview(contacts_df, "contacts_df")

Overview statistics for contacts_df:
Number of rows: 27887
Number of columns: 14
Number of rows with null values in any column: 16300
% of rows with null values in any column: 58.45%
Number of null cells: 29805
% of cells with null values: 7.63%


Below is a visual printout of the first 5 rows of the data within *contacts_df*. To present the data in a more readable matter, the data was presented in a transposed format, where each row below is a data field and each column is an inquiry.

Upon visual examination, the 3 data fields that begin with *'id'* are useful for identifying unique inquiries within *contacts_df* and are useful for connecting to supporting data tables *listings_df* and *users_df*.

Some cells below were observed to contain 'NaT', which stands for *Not a Time*. This means that there were null values or objects in the cells that cannot be recognized as datetime objects by the pandas module. However, this is not a problem in the data and should instead be understood as a inquiry that did not become a booking or an `instant_book` inquiry, in which no response or confirmation is needed from the host.

In [19]:
# Visually examine first 5 rows of data
display(contacts_df.head().T)

,0,1,2,3,4
id_guest_anon,da8656a1-51af-4f38-b1c4-94be1f585157,8590d6f1-8bc9-4e8b-bdfb-de78f69fcf37,ebcd83ba-bda1-47eb-9680-2dd04ccf3988,b0af8848-fe2a-4ef1-991e-26ab3066feb3,5ddbbcc3-ac1a-4d8a-b6f8-0f675b76d1b2
id_host_anon,5426897d-960d-4013-9e38-606ae746793c,f30417c5-6df4-45ac-bfc2-6ad1cce398ab,13cbf50a-3272-45d4-9866-a06b6ea1b99a,01614601-d5a4-4776-ab9b-c10d3b865bf0,f2fed6f3-4c5c-453d-9e64-37c62b8bd06d
id_listing_anon,a408a8b2-0d44-4513-a611-3736d0409bb2,e387c705-0aeb-464c-9375-ece63be6f006,d1eb1960-938f-4305-a353-51e224414dd4,855f6779-346c-45fc-a64b-ea133250ca7a,f2928a59-c5e7-42b2-9c37-240a40f38dc0
ts_interaction_first,2016-04-21 02:55:53,2016-02-16 22:14:01,2016-01-27 23:33:38,2016-05-05 14:42:52,2016-06-23 03:09:25
ts_reply_at_first,2016-04-21 03:15:00,2016-02-16 23:37:36,2016-01-28 02:12:47,2016-05-05 15:17:40,2016-06-23 03:09:26
ts_accepted_at_first,2016-04-21 03:15:00,NaT,NaT,2016-05-05 15:17:40,2016-06-23 03:09:33
ts_booking_at,2016-04-21 03:15:00,NaT,NaT,NaT,2016-06-23 03:09:33
ds_checkin_first,2016-08-02 00:00:00,2016-08-11 00:00:00,2016-03-14 00:00:00,2016-05-27 00:00:00,2016-08-19 00:00:00
ds_checkout_first,2016-08-06 00:00:00,2016-08-22 00:00:00,2016-03-23 00:00:00,2016-05-29 00:00:00,2016-08-21 00:00:00
m_guests,4.0,5.0,1.0,2.0,3.0


In [27]:
# Create a combination of the three id columns to check for unique values
contacts_df['uid'] = contacts_df['id_guest_anon'] + contacts_df['id_host_anon'] + contacts_df['id_listing_anon']

if contacts_df['uid'].nunique() == contacts_df.shape[0]:
    print("The combination of id columns form a unique identifier for each AirBnB inquiry.")

In [28]:
contacts_df['uid'].nunique()

27788

In [29]:
contacts_df.shape[0]

27887

### Supporting data tables

Two supporting data files: `Listings.csv` and `Users.csv` were provided. Similar to `contacts_df`, the 

In [16]:
# Importing the data for every listing
listings_df = pd.read_csv('data/listings.csv')

**Listings.csv** contains data for every listing in the market.

In [17]:
# Examine the overview for the loaded data
dataframe_overview(listings_df, "listings_df")

Overview statistics for listings_df:
Number of rows: 13038
Number of columns: 4
Number of rows with null values in any column: 0
% of rows with null values in any column: 0.0%
Number of null cells: 0
% of cells with null values: 0.0%


In [8]:
contacts_df.isna().any(axis = 1).sum()

16300

In [ ]:
contacts_df.

In [ ]:
contacts_df.info()